In [1]:
import pandas as pd

In [10]:
df = pd.read_csv('C:/Users/rafaR/Documents/DataS/proyectos/diabetes.csv')

In [12]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
from sklearn.model_selection import GridSearchCV#permite crear la rejilla(grid) de busqueda con todas ls combinianciones posibles de los hiperparamtros
from sklearn.neighbors import KNeighborsClassifier#nuestro modelo
from sklearn.pipeline import Pipeline#nos ayuda preprocesamiento, selección de características, entrenamiento y evaluación
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
X, y =df